In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait       
from selenium.webdriver.common.by import By       
from selenium.webdriver.support import expected_conditions as EC

options = Options()
options.headless = True

import numpy as np
import pandas as pd
import time as time
from time import sleep
import random
from tqdm import tqdm
from IPython.display import clear_output

In [2]:
from nba_api.stats.endpoints import leaguegamelog
from nba_api.stats.library.parameters import SeasonAll
from nba_api.stats.static import teams

In [4]:
def season_string(season):
    return str(season) + '-' + str(season+1)[-2:]

def get_game_dates(season):
    season_str = season_string(season)
    dates = []
    for season_type in ['Regular Season', 'Playoffs']:
        games = leaguegamelog.LeagueGameLog(season=season_str, season_type_all_star=season_type).get_data_frames()[0]
        dates.extend(games['GAME_DATE'].unique())
        sleep(1)
    return dates


get_game_dates(2023)

['2023-10-24',
 '2023-10-25',
 '2023-10-26',
 '2023-10-27',
 '2023-10-28',
 '2023-10-29',
 '2023-10-30',
 '2023-10-31',
 '2023-11-01',
 '2023-11-02',
 '2023-11-03',
 '2023-11-04',
 '2023-11-05',
 '2023-11-06',
 '2023-11-08',
 '2023-11-09',
 '2023-11-10',
 '2023-11-11',
 '2023-11-12',
 '2023-11-13',
 '2023-11-14',
 '2023-11-15',
 '2023-11-16',
 '2023-11-17',
 '2023-11-18',
 '2023-11-19',
 '2023-11-20',
 '2023-11-21',
 '2023-11-22',
 '2023-11-24',
 '2023-11-25',
 '2023-11-26',
 '2023-11-27',
 '2023-11-28',
 '2023-11-29',
 '2023-11-30',
 '2023-12-01',
 '2023-12-02',
 '2023-12-04',
 '2023-12-05',
 '2023-12-06',
 '2023-12-07']

In [5]:
get_game_dates(2023)[:8]

['2023-10-24',
 '2023-10-25',
 '2023-10-26',
 '2023-10-27',
 '2023-10-28',
 '2023-10-29',
 '2023-10-30',
 '2023-10-31']

In [7]:
dates = get_game_dates(2023)[:8]
dates_with_no_data = []
seasons = []
gm_dates = []
away_teams = []
home_teams = []
away_mls = []
home_mls = []      
for date in tqdm(dates, desc='progress'):
    web = 'https://www.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date={}'.format(date)
    # path = '../chromedriver.exe'
    driver = webdriver.Chrome()
    driver.get(web)
    sleep(random.randint(1,2))

    try:
        single_row_events = driver.find_elements_by_class_name('eventMarketGridContainer-3QipG')
    
    except:
        print("No Data for {}".format(date))
        dates_with_no_data.append(date)
        continue
                
    num_postponed_events = len(driver.find_elements_by_class_name('eventStatus-3EHqw'))

    num_listed_events = len(single_row_events)
    cutoff = num_listed_events - num_postponed_events

    for event in single_row_events[:cutoff]:
        seasons.append(season_string(2023))

        away_team = event.find_elements_by_class_name('participantBox-3ar9Y')[0].text
        home_team = event.find_elements_by_class_name('participantBox-3ar9Y')[1].text

        away_teams.append(away_team)
        home_teams.append(home_team)

        gm_dates.append(date)

        mls = event.find_elements_by_class_name('pointer-2j4Dk')
        
        away_moneyline = []
        home_moneyline = []
        
        
        for i, ml in enumerate(mls):
            if i%2==0:
                away_moneyline.append(ml.text)
            else:
                home_moneyline.append(ml.text)
        
        away_moneyline = ",".join(away_moneyline)
        home_moneyline = ",".join(home_moneyline)

        away_mls.append(away_moneyline)
        home_mls.append(home_moneyline)
        
    driver.quit()
    
clear_output(wait=True)

df = pd.DataFrame({'SEASON':seasons,
                'GM_DATE':gm_dates,
                'AWAY_TEAM':away_teams, 
                'HOME_TEAM':home_teams,
                'AWAY_ML':away_mls,
                'HOME_ML':home_mls,
                                    })

progress:  12%|█▎        | 1/8 [00:08<00:57,  8.17s/it]

No Data for 2023-10-24


progress:  25%|██▌       | 2/8 [00:13<00:40,  6.79s/it]

No Data for 2023-10-25


progress:  38%|███▊      | 3/8 [00:19<00:30,  6.14s/it]

...

No Data for 2023-10-29


progress:  88%|████████▊ | 7/8 [00:38<00:05,  5.00s/it]

No Data for 2023-10-30


progress: 100%|██████████| 8/8 [00:44<00:00,  5.53s/it]

No Data for 2023-10-31


In [8]:
df

,SEASON,GM_DATE,AWAY_TEAM,HOME_TEAM,AWAY_ML,HOME_ML
